In [1]:
# imports
import pandas as pd
import math
import zipfile
from credentials import api_key
import yfinance as yf

In [2]:
def get_key_metrics(zip_file_path, ticker):
    file_name = f'{ticker}_key-metrics.json'
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        with zip_ref.open(file_name) as file:
            content = pd.read_json(file)
            return content


In [71]:
from datetime import datetime

# util function (source: https://stackoverflow.com/questions/32237862/find-the-closest-date-to-a-given-date)
def get_nearest_date(items,pivot):
    # Convert the date strings to datetime objects
    items = [datetime.strptime(date, "%Y-%m-%d") for date in items]
    pivot = datetime.strptime(pivot, "%Y-%m-%d")

    nearest=min(items, key=lambda x: abs(x - pivot))
    timedelta = abs(nearest - pivot)
    return nearest, timedelta


def get_closing_price(ticker, target_date):

    closing_price = None

    alpha_vantage_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&outputsize=full&apikey={api_key}&datatype=csv'
    alpha_vantage_csv = pd.read_csv(alpha_vantage_url)
    alpha_vantage_df = pd.DataFrame(alpha_vantage_csv)

    # Try fetching from Alpha Vantage
    for index, row in alpha_vantage_df.iterrows():
        if row['timestamp'] == target_date:
            closing_price = row['close']
            # print(f"Found exact match in Alpha Vantage: {closing_price}")
            return closing_price

    if closing_price is None:
        # If Alpha Vantage fails, try fetching from Yahoo Finance
        yfinance_ticker = yf.Ticker(ticker)
        historical_data = yfinance_ticker.history(period="25y")

        for index, row in historical_data.iterrows():
            date = index.strftime("%Y-%m-%d")
            if date == target_date:
                closing_price = row['Close']
                # print(f"Found exact match in Yahoo Finance: {closing_price}")
                return closing_price

    if closing_price is None:
        # If both fail, find the nearest closing price
        dates = alpha_vantage_df['timestamp']
        nearest_date, timedelta = get_nearest_date(dates, target_date)

        # Get closing price of nearest date
        nearest_date_str = nearest_date.strftime("%Y-%m-%d")
        for index, row in alpha_vantage_df.iterrows():
            if row['timestamp'] == nearest_date_str:
                nearest_closing_price = row['close']
                # print(f"Found nearest closing price: {nearest_closing_price} on {nearest_date_str}")
                return nearest_closing_price
    else:
        return closing_price

In [72]:
# function to get the reported day for a specific quarter of a company

def get_reported_date(ticker, target_quarter, target_year):
    # Get key metrics
    key_metrics = get_key_metrics('key-metrics_3-Aktien.zip', ticker)
    for index, row in key_metrics.iterrows():
        quarter = row['period']
        year = str(row['calendarYear'])

        if quarter == target_quarter and year == target_year:
            target_date = row['date'].strftime("%Y-%m-%d")
            return target_date


print(get_reported_date('AAPL', 'Q1', '2023'))

2022-12-31


In [ ]:
ticker = "AAPL"
target_date = '2002-06-30'
closing_price = get_closing_price(ticker, target_date)
print(closing_price)


In [3]:
def calculate_graham_number(ticker, target_quarter, target_year):
    # Get key metrics
    key_metrics = get_key_metrics('key-metrics_3-Aktien.zip', ticker)
    for index, row in key_metrics.iterrows():
        quarter = row['period']
        year = str(row['calendarYear'])

        if quarter == target_quarter and year == target_year:
            eps = row['netIncomePerShare']
            bvps = row['bookValuePerShare']

            # Check if eps and bvps are non-negative before calculating Graham number
            if eps >= 0 and bvps >= 0:
                graham_number = math.sqrt(22.5 * eps * bvps)
                # print(f'The calculated Graham number on {target_date} is {graham_number}')

            else:
                graham_number = row['grahamNumber']
                # print("EPS or BVPS is negative. Took the given Graham Number instead of calculating it.")

            return graham_number

In [7]:
# calculate price by ratio
def price_by_ratio(ticker, target_quarter, target_year):
    # Get key metrics
    key_metrics = get_key_metrics('key-metrics_3-Aktien.zip', ticker)
    for index, row in key_metrics.iterrows():
        quarter = row['period']
        year = str(row['calendarYear'])

        if quarter == target_quarter and year == target_year:
            date = row['date'].strftime("%Y-%m-%d")
            netIncomePerShare = row['netIncomePerShare']
            pe_ratio = row['peRatio']

            price = pe_ratio * netIncomePerShare

            print(f'price on {date} is {price}')

            return price

print(price_by_ratio('AAPL', 'Q4', '2021'))

price on 2021-09-25 is 36.72999999999998
36.72999999999998


In [78]:
def get_margin_of_safety(ticker, target_quarter, target_year):

    target_date = get_reported_date(ticker, target_quarter, target_year)
    closing_price = get_closing_price(ticker, target_date)
    graham_number = calculate_graham_number(ticker, target_quarter, target_year)

    # find the undervalued stocks
    if graham_number is not None and closing_price is not None:
        margin_of_safety = graham_number - closing_price
        return margin_of_safety
    else:
        return 'some values are missing'

In [81]:
years = ['2000', '2001', '2002'#, '2003', '2004', '2005',
         #'2006', '2007', '2008', '2009', '2010', '2011',
         #'2012', '2013', '2014', '2015', '2016', '2017'
         ]
quarters = ['Q1', 'Q2', 'Q3', 'Q4']


ticker_list = ['AAPL', 'MSFT', 'PEP']
for ticker in ticker_list:
    for year in years:
        for quarter in quarters:
            portfolio = []
            margin = get_margin_of_safety(ticker, quarter, year)
            if margin > 0:
                portfolio.append(ticker)
                sorted_portfolio = sorted(portfolio, key=lambda x: x[1], reverse=True)
                print(f'{quarter}{year}: {sorted_portfolio}')
            else:
                print(f'In {quarter}, {year}, {ticker} is overvalued')



In Q22001, AAPL is overvalued
In Q32001, AAPL is overvalued
In Q42001, AAPL is overvalued
In Q12002, AAPL is overvalued
In Q22002, AAPL is overvalued
In Q32002, AAPL is overvalued
In Q42002, AAPL is overvalued
In Q12000, MSFT is overvalued
In Q22000, MSFT is overvalued
In Q32000, MSFT is overvalued
In Q42000, MSFT is overvalued
In Q12001, MSFT is overvalued
In Q22001, MSFT is overvalued
In Q32001, MSFT is overvalued
In Q42001, MSFT is overvalued
In Q12002, MSFT is overvalued
In Q22002, MSFT is overvalued
In Q32002, MSFT is overvalued
In Q42002, MSFT is overvalued
In Q12000, PEP is overvalued
In Q22000, PEP is overvalued
In Q32000, PEP is overvalued
In Q42000, PEP is overvalued
In Q12001, PEP is overvalued
In Q22001, PEP is overvalued
In Q32001, PEP is overvalued
In Q42001, PEP is overvalued
In Q12002, PEP is overvalued
In Q22002, PEP is overvalued
In Q32002, PEP is overvalued
In Q42002, PEP is overvalued
In Q1, 2000, AAPL is overvalued
In Q2, 2000, AAPL is overvalued
In Q3, 2000, AAPL 